In [138]:
import numpy as np
import pandas as pd

import os
import re
import time
import math

from sklearn.preprocessing import LabelEncoder

In [2]:
!ls

Data_Q8       Q8.ipynb      Q8_code.ipynb readme.md


In [3]:
data_root = './Data_Q8/'

In [4]:
user_df = pd.read_csv(os.path.join(data_root, 'users.csv'))
movie_df = pd.read_csv(os.path.join(data_root, 'movies.csv'))
rating_df = pd.read_csv(os.path.join(data_root, 'rating_train.csv'))

## Data analysis & Feature Engineering

In [87]:
user_df.head()

,UserID,Gender,Age,OccupationID,Zip-code,F,M,Zip3,zip_label
0,1,F,1,10,48067,1,0,480,348
1,2,M,7,16,70072,0,1,700,485
2,3,M,3,15,55117,0,1,551,400
3,4,M,5,7,02460,0,1,024,17
4,5,M,3,20,55455,0,1,554,402


In [55]:
movie_df.head()

,MovieID,Title,Genres,publish_time,Action,Adventure,Animation,Children's,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,avg_rating,rating_F,rating_M
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,0,0,1,1,1,0,...,0,0,0,0,0,0,0,4.138812,4.118774,4.191532
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,0,1,0,1,0,0,...,0,0,0,0,0,0,0,3.222581,3.194805,3.090164
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,0,0,0,0,1,0,...,0,0,1,0,0,0,0,3.023148,2.996774,3.224490
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,2.719178,2.460526,3.585635
4,5,Father of the Bride Part II (1995),Comedy,1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,3.029963,2.917160,2.375000


In [56]:
rating_df.head()

,UserID,MovieID,timestamps,Rating,rate_year,rate_month,rate_day
0,1,1836,978300172,5,2001,01,01
1,1,1097,978301953,4,2001,01,01
2,1,2028,978301619,5,2001,01,01
3,1,527,978824195,5,2001,01,07
4,1,2918,978302124,4,2001,01,01


#### User based

In [8]:
user_df['Age'] = LabelEncoder().fit_transform(user_df['Age'])+1

In [9]:
user_df = user_df.join(pd.get_dummies(user_df['Gender']))

In [10]:
ocup = {
    0: 'other or not specified',
    1: 'academic/educator',
    2: 'artist',
    3: 'clerical/admin',
    4: 'college/grad student',
    5: 'customer service',
    6: 'doctor/health care',
    7: 'executive/managerial',
    8: 'farmer',
    9: 'homemaker',
    10: 'K-12 student',
    11: 'lawyer',
    12: 'programmer',
    13: 'retired',
    14: 'sales/marketing',
    15: 'scientist',
    16: 'self-employed',
    17: 'technician/engineer',
    18: 'tradesman/craftsman',
    19: 'unemployed',
    20: 'writer'
}

In [79]:
user_df['Zip3'] = user_df['Zip-code'].map(lambda x: x[:3])
user_df['zip_label'] = LabelEncoder().fit_transform(user_df['Zip3'])

In [80]:
len(user_df['zip_label'].unique())

678

#### Movie based

In [12]:
pattern = re.compile('\d+')
movie_df['publish_time'] = movie_df['Title'].map(lambda x: pattern.findall(x)[-1])

In [13]:
types = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary',
         'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
         'Thriller', 'War', 'Western']
for g in types:
    movie_df[g] = 0
    movie_df[g]=movie_df[g].mask(np.array([g in gen.split('|') for gen in movie_df['Genres']]), 1)

#### relations

In [14]:
rating_df['rate_year'] = rating_df['timestamps'].map(lambda x: time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(x)))

rating_df['rate_month'] = rating_df['rate_year'].map(lambda x: x.split('-')[1])
rating_df['rate_day'] = rating_df['rate_year'].map(lambda x: x.split('-')[2])
rating_df['rate_year'] = rating_df['rate_year'].map(lambda x: x.split('-')[0])

In [16]:
temp = pd.merge(rating_df, user_df[['UserID','Age','OccupationID','Zip-code','F','M']], how='left', on='UserID')

In [17]:
temp

,UserID,MovieID,timestamps,Rating,rate_year,rate_month,rate_day,Age,OccupationID,Zip-code,F,M
0,1,1836,978300172,5,2001,01,01,1,10,48067,1,0
1,1,1097,978301953,4,2001,01,01,1,10,48067,1,0
2,1,2028,978301619,5,2001,01,01,1,10,48067,1,0
3,1,527,978824195,5,2001,01,07,1,10,48067,1,0
4,1,2918,978302124,4,2001,01,01,1,10,48067,1,0
5,1,2692,978301570,4,2001,01,01,1,10,48067,1,0
6,1,260,978300760,4,2001,01,01,1,10,48067,1,0
7,1,938,978301752,4,2001,01,01,1,10,48067,1,0
8,1,1022,978300055,5,2001,01,01,1,10,48067,1,0
9,1,1207,978300719,4,2001,01,01,1,10,48067,1,0


In [50]:
temp_movie = temp.groupby('MovieID').agg({
    'Rating': 'mean'
})
temp_movie = temp_movie.reset_index().rename(index=str, columns={'Rating': 'avg_rating'})

movie_df = pd.merge(movie_df, temp_movie, how='left', on='MovieID')

In [51]:
temp_movie = temp.groupby(['MovieID', 'F']).agg({
    'Rating': 'mean'
})

In [52]:
temp_movie = temp_movie.reset_index()
emp_df = temp_movie[['MovieID']].drop_duplicates(subset=['MovieID'], keep='first')

emp_df['rating_F'] = temp_movie.query('F=="0"')['Rating']
emp_df['rating_M'] = temp_movie.query('F=="1"').reset_index()['Rating']

In [54]:
# temp_movie = temp_movie.reset_index().rename(index=str, columns={'Rating': 'avg_rating'})
movie_df = pd.merge(movie_df, emp_df, how='left', on='MovieID')

In [84]:
rating_df

,UserID,MovieID,timestamps,Rating,rate_year,rate_month,rate_day
0,1,1836,978300172,5,2001,01,01
1,1,1097,978301953,4,2001,01,01
2,1,2028,978301619,5,2001,01,01
3,1,527,978824195,5,2001,01,07
4,1,2918,978302124,4,2001,01,01
5,1,2692,978301570,4,2001,01,01
6,1,260,978300760,4,2001,01,01
7,1,938,978301752,4,2001,01,01
8,1,1022,978300055,5,2001,01,01
9,1,1207,978300719,4,2001,01,01


In [57]:
movie_df

,MovieID,Title,Genres,publish_time,Action,Adventure,Animation,Children's,Comedy,Crime,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,avg_rating,rating_F,rating_M
0,1,Toy Story (1995),Animation|Children's|Comedy,1995,0,0,1,1,1,0,...,0,0,0,0,0,0,0,4.138812,4.118774,4.191532
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1995,0,1,0,1,0,0,...,0,0,0,0,0,0,0,3.222581,3.194805,3.090164
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,0,0,0,0,1,0,...,0,0,1,0,0,0,0,3.023148,2.996774,3.224490
3,4,Waiting to Exhale (1995),Comedy|Drama,1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,2.719178,2.460526,3.585635
4,5,Father of the Bride Part II (1995),Comedy,1995,0,0,0,0,1,0,...,0,0,0,0,0,0,0,3.029963,2.917160,2.375000
5,6,Heat (1995),Action|Crime|Thriller,1995,1,0,0,0,0,1,...,0,0,0,0,1,0,0,3.884248,3.915395,3.924915
6,7,Sabrina (1995),Comedy|Romance,1995,0,0,0,0,1,0,...,0,0,1,0,0,0,0,3.409756,3.270742,3.666667
7,8,Tom and Huck (1995),Adventure|Children's,1995,0,1,0,1,0,0,...,0,0,0,0,0,0,0,2.967213,2.666667,3.428571
8,9,Sudden Death (1995),Action,1995,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2.666667,2.694118,4.229607
9,10,GoldenEye (1995),Action|Adventure|Thriller,1995,1,1,0,0,0,0,...,0,0,0,0,1,0,0,3.527457,3.544992,2.269231


#### Combination

In [60]:
user_df.columns

Index(['UserID', 'Gender', 'Age', 'OccupationID', 'Zip-code', 'F', 'M'], dtype='object')

In [88]:
mov_cols = ['MovieID', 'publish_time', 'Action', 'Adventure',
           'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama',
           'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
           'Sci-Fi', 'Thriller', 'War', 'Western', 'avg_rating', 'rating_F', 'rating_M']
usr_cols = ['UserID', 'Age', 'OccupationID', 'zip_label', 'F', 'M']

In [89]:
rating_df = pd.merge(rating_df[['UserID','MovieID','timestamps','Rating','rate_year']], user_df[usr_cols], how='left', on='UserID')
rating_df = pd.merge(rating_df, movie_df[mov_cols], how='left', on='MovieID')

In [99]:
medium_rf = rating_df['rating_F'].quantile(0.5)
medium_rm = rating_df['rating_M'].quantile(0.5)

rating_df[['rating_F']] = rating_df[['rating_F']].fillna(medium_rf)
rating_df[['rating_M']] = rating_df[['rating_M']].fillna(medium_rm)

In [110]:
rating_df[['avg_rating']].isna().any()

avg_rating    False
dtype: bool

In [111]:
rating_df['rating_F'] = rating_df['rating_F']*rating_df['F']
rating_df['rating_M'] = rating_df['rating_M']*rating_df['M']

In [119]:
rating_df['rate_year'] = rating_df['rate_year'].astype(int)
rating_df['publish_time'] = rating_df['publish_time'].astype(int)

### Modeling

In [158]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error

In [159]:
train_x = rating_df.drop(['UserID','MovieID','Rating','timestamps'], axis=1)
train_y = rating_df['Rating']

In [160]:
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)

In [161]:
xgb = XGBRegressor(booster='gblinear', n_estimators=500, max_depth=3)
pred_y = xgb.fit(x_train, y_train).predict(x_test)
pred_y = [round(x) if x < 5.5 else 5 for x in pred_y]

print('accruracy =', accuracy_score(y_test, pred_y))
print('rmse',math.sqrt(mean_absolute_error(y_test, pred_y)))

accruracy = 0.37295909122151477
rmse 0.878524685128167


In [162]:
xgb = XGBRegressor(booster='gbtree', n_estimators=600, max_depth=4)
pred_y = xgb.fit(x_train, y_train).predict(x_test)
pred_y = [round(x) if x < 5.5 else 5 for x in pred_y]
print('accruracy =', accuracy_score(y_test, pred_y))
print('rmse',math.sqrt(mean_absolute_error(y_test, pred_y)))

accruracy = 0.39298545979417515
rmse 0.84901372137151


In [155]:
train_x.head()

,UserID,MovieID,timestamps,rate_year,Age,OccupationID,zip_label,F,M,publish_time,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,avg_rating,rating_F,rating_M
0,1,1836,978300172,2001,1,10,348,1,0,1998,...,0,0,0,0,0,0,0,3.242718,3.191781,0.0
1,1,1097,978301953,2001,1,10,348,1,0,1982,...,0,0,0,1,0,0,0,3.954650,3.903698,0.0
2,1,2028,978301619,2001,1,10,348,1,0,1998,...,0,0,0,0,0,1,0,4.323810,4.388424,0.0
3,1,527,978824195,2001,1,10,348,1,0,1993,...,0,0,0,0,0,1,0,4.506593,4.486204,0.0
4,1,2918,978302124,2001,1,10,348,1,0,1986,...,0,0,0,0,0,0,0,4.108392,4.124113,0.0


### Output

In [163]:
test_df = pd.read_csv(os.path.join(data_root, 'rating_test.csv'))

In [164]:
test_df['rate_year'] = test_df['timestamps'].map(lambda x: time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(x)))
test_df['rate_year'] = test_df['rate_year'].map(lambda x: x.split('-')[0])

In [165]:
test_df = pd.merge(test_df[['UserID','MovieID','timestamps','rate_year']], user_df[usr_cols], how='left', on='UserID')
test_df = pd.merge(test_df, movie_df[mov_cols], how='left', on='MovieID')

medium_rf = rating_df['rating_F'].quantile(0.5)
medium_rm = rating_df['rating_M'].quantile(0.5)

test_df[['rating_F']] = test_df[['rating_F']].fillna(medium_rf)
test_df[['rating_M']] = test_df[['rating_M']].fillna(medium_rm)

In [166]:
test_df['rating_F'] = test_df['rating_F']*rating_df['F']
test_df['rating_M'] = test_df['rating_M']*rating_df['M']

In [167]:
test_df['rate_year'] = test_df['rate_year'].astype(int)
test_df['publish_time'] = test_df['publish_time'].astype(int)

In [168]:
test_df.columns

Index(['UserID', 'MovieID', 'timestamps', 'rate_year', 'Age', 'OccupationID',
       'zip_label', 'F', 'M', 'publish_time', 'Action', 'Adventure',
       'Animation', 'Children's', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Thriller', 'War', 'Western', 'avg_rating', 'rating_F',
       'rating_M'],
      dtype='object')

In [173]:
test_x = test_df.drop(['UserID','MovieID','timestamps'],axis=1)
xgb = XGBRegressor(booster='gbtree', n_estimators=600, max_depth=3)
xgb.fit(train_x, train_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [174]:
pred_y = xgb.predict(test_x)
pred_y = [round(x) if x < 5.5 else 5 for x in pred_y]
test_df['Rating'] = pred_y

In [175]:
test_df[['UserID', 'MovieID', 'Rating']].to_csv('./Q8_output.csv')